# bicubic interpolation 

In [265]:
import numpy as np
from scipy.interpolate import RectBivariateSpline, CubicSpline
from numba import njit, jit

# Example 2D grid (x, y) and corresponding function values z
x = np.linspace(0., 5, 100)
y = np.linspace(0., 5, 100)
x_, y_ = np.meshgrid(x, y)
z = np.sin(x_**2 + y_**2)  # Example function values
fn_ = lambda x_,y_: np.sin(x_**2 + y_**2)

* RectBivariateSpline needs size of x and y the same

In [266]:
interp_spline = RectBivariateSpline(x_[0, :], y_[:,0], z)

In [267]:
x_new = 4
y_new = 4
interp_spline(x_new, y_new)

array([[0.55138675]])

In [268]:
fn_(x_new, y_new)

0.5514266812416906

In [43]:
x_new = np.random.uniform(0, 5, 25000)
y_new = np.random.uniform(0, 5, 25000)

def interp(x_new, y_new):
    z_new = np.zeros_like(x_new)
    for i in range(len(x_new)):
        z_new[i] = interp_spline(x_new[i], y_new[i])
    return z_new

In [46]:
%time interp(x_new, y_new)

CPU times: user 47.6 ms, sys: 2.47 ms, total: 50.1 ms
Wall time: 48.6 ms


array([ 0.40865133, -0.79037589,  0.91349236, ...,  0.7891974 ,
       -0.41989411, -0.34900501])

In [269]:
x = np.linspace(0, 5, 100)
y = np.linspace(0, 5, 100)
fn_ = lambda x_,y_: np.sin(x_**2 + y_**2)

In [270]:
y_, x_ = np.meshgrid(y, x)
z = fn_(x_, y_)
print(np.shape(z))

(100, 100)


In [271]:
# yfixed
coeff_yfixed = []
derivatives1_yfixed = []
derivatives2_yfixed = []
for i in range(len(y)):
    spline = CubicSpline(x, z[:, i])
    coeff_yfixed.append(spline.c)
    derivatives1_yfixed.append(spline.derivative(1)(x))
    derivatives2_yfixed.append(spline.derivative(2)(x))
coeff_yfixed = np.array(coeff_yfixed)
derivatives1_yfixed = np.array(derivatives1_yfixed)
derivatives2_yfixed = np.array(derivatives2_yfixed)

In [272]:
# xfixed
coeff_xfixed = []
derivatives1_xfixed = []
derivatives2_xfixed = []
for i in range(len(x)):
    spline = CubicSpline(y, z[i, :])
    coeff_xfixed.append(spline.c)
    derivatives1_xfixed.append(spline.derivative(1)(y))
    derivatives2_xfixed.append(spline.derivative(2)(y))
coeff_xfixed = np.array(coeff_xfixed)
derivatives1_xfixed = np.array(derivatives1_xfixed)
derivatives2_xfixed = np.array(derivatives2_xfixed)

In [273]:
print(np.shape(derivatives1_yfixed))
print(np.shape(derivatives1_xfixed))

(100, 100)
(100, 100)


In [274]:
# partial derivatives at each [x,y] points
partial_derivatives1 = np.zeros((len(x), len(y), 2))
partial_derivatives2 = np.zeros((len(x), len(y), 2))
for i in range(len(x)):
    for j in range(len(y)):
        partial_derivatives1[i, j,0] = derivatives1_yfixed[j, i]
        partial_derivatives1[i, j, 1] = derivatives1_xfixed[i, j]
        partial_derivatives2[i, j,0] = derivatives2_yfixed[j, i]
        partial_derivatives2[i, j, 1] = derivatives2_xfixed[i, j]

In [275]:
print(np.shape(partial_derivatives1))

(100, 100, 2)


In [276]:
print(x)
print(y)
print(z)

[0.         0.05050505 0.1010101  0.15151515 0.2020202  0.25252525
 0.3030303  0.35353535 0.4040404  0.45454545 0.50505051 0.55555556
 0.60606061 0.65656566 0.70707071 0.75757576 0.80808081 0.85858586
 0.90909091 0.95959596 1.01010101 1.06060606 1.11111111 1.16161616
 1.21212121 1.26262626 1.31313131 1.36363636 1.41414141 1.46464646
 1.51515152 1.56565657 1.61616162 1.66666667 1.71717172 1.76767677
 1.81818182 1.86868687 1.91919192 1.96969697 2.02020202 2.07070707
 2.12121212 2.17171717 2.22222222 2.27272727 2.32323232 2.37373737
 2.42424242 2.47474747 2.52525253 2.57575758 2.62626263 2.67676768
 2.72727273 2.77777778 2.82828283 2.87878788 2.92929293 2.97979798
 3.03030303 3.08080808 3.13131313 3.18181818 3.23232323 3.28282828
 3.33333333 3.38383838 3.43434343 3.48484848 3.53535354 3.58585859
 3.63636364 3.68686869 3.73737374 3.78787879 3.83838384 3.88888889
 3.93939394 3.98989899 4.04040404 4.09090909 4.14141414 4.19191919
 4.24242424 4.29292929 4.34343434 4.39393939 4.44444444 4.4949

In [277]:
# vectorization
# bicubic interpolation
x_new = 4
y_new = 4
# search for the nearest (but above) x and y
x_idx = np.searchsorted(x, x_new)-1
y_idx = np.searchsorted(y, y_new)-1

In [278]:
print(x_idx)
print(y_idx)

79
79


In [279]:
x_idx1 = x_idx+1
x_idx2 = x_idx+1
x_idx3 = x_idx
x_idx4 = x_idx
y_idx1 = y_idx+1
y_idx2 = y_idx
y_idx3 = y_idx+1
y_idx4 = y_idx
x1, x2, x3, x4 = x[x_idx1], x[x_idx2], x[x_idx3], x[x_idx4]
y1, y2, y3, y4 = y[y_idx1], y[y_idx2], y[y_idx3], y[y_idx4]

x_idx1_ = x_idx1-1
x_idx2_ = x_idx2-1
x_idx3_ = x_idx3-1
x_idx4_ = x_idx4-1

matrixA = np.array([
    [x1**3, x1**2, x1, y1**3, y1**2, y1, x1**2*y1, x1*y1**2, x1*y1, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [x2**3, x2**2, x2, y1**3, y1**2, y1, x2**2*y1, x2*y1**2, x2*y1, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,x2**3, x2**2, x2, y1**3, y1**2, y1, x2**2*y1, x2*y1**2, x2*y1, 1,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,x3**3, x3**2, x3, y1**3, y1**2, y1, x3**2*y1, x3*y1**2, x3*y1, 1,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,x3**3, x3**2, x3, y1**3, y1**2, y1, x3**2*y1, x3*y1**2, x3*y1, 1],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,x4**3, x4**2, x4, y1**3, y1**2, y1, x4**2*y1, x4*y1**2, x4*y1],

    
    ])

/var/folders/ws/0948zvwd7g795j2l3fryghjw0000gp/T/ipykernel_99764/1700135318.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  matrixA = np.array([


In [280]:
matrixA = np.array([
    [x1**3, x1**2, x1, y1**3, y1**2, y1, x1**2*y1, x1*y1**2, x1*y1, 1],
    [x2**3, x2**2, x2, y2**3, y2**2, y2, x2**2*y2, x2*y2**2, x2*y2, 1],
    [x3**3, x3**2, x3, y3**3, y3**2, y3, x3**2*y3, x3*y3**2, x3*y3, 1],
    [x4**3, x4**2, x4, y4**3, y4**2, y4, x4**2*y4, x4*y4**2, x4*y4, 1],
    [3*x1**2, 2*x1, 1,0,0,0, 2*x1*y1, y1**2, y1,0],
    [3*x2**2, 2*x2, 1,0,0,0, 2*x2*y2, y2**2, y2,0],
    [3*x4**2, 2*x4, 1,0,0,0, 2*x4*y4, y4**2, y4,0],
    [0,0,0, 3*y1**2, 2*y1, 1, x1**2, 2*x1*y1, x1,0],
    [0,0,0, 3*y2**2, 2*y2, 1, x2**2, 2*x2*y2, x2,0],
    [0,0,0, 3*y4**2, 2*y4, 1, x4**2, 2*x4*y4, x4,0],
    ])

In [281]:
# vectorization
x_new = 4
y_new = 4
# search for the nearest (but above) x and y
x_idx = np.searchsorted(x, x_new)
y_idx = np.searchsorted(y, y_new)
# bicubic interpolation
x_idx1 = x_idx-1
x_idx2 = x_idx-1
x_idx3 = x_idx
x_idx4 = x_idx
y_idx1 = y_idx-1
y_idx2 = y_idx
y_idx3 = y_idx-1
y_idx4 = y_idx
x1, x2, x3, x4 = x[x_idx1], x[x_idx2], x[x_idx3], x[x_idx4]
y1, y2, y3, y4 = y[y_idx1], y[y_idx2], y[y_idx3], y[y_idx4]

matrixA = np.array([
    [x4**3, x4**2, x4, y4**3, y4**2, y4, x4**2*y4, x4*y4**2, x4*y4, 1],
    [3*x1**2, 2*x1, 1,0,0,0, 2*x1*y1, y1**2, y1,0],
    [3*x2**2, 2*x2, 1,0,0,0, 2*x2*y2, y2**2, y2,0],
    [3*x3**2, 2*x3, 1,0,0,0, 2*x3*y3, y3**2, y3,0],
    [0,0,0, 3*y1**2, 2*y1, 1, x1**2, 2*x1*y1, x1,0],
    [0,0,0, 3*y2**2, 2*y2, 1, x2**2, 2*x2*y2, x2,0],
    [0,0,0, 3*y3**2, 2*y3, 1, x3**2, 2*x3*y3, x3,0],
    [6*x1, 2, 0,0,0,0, 2*y1, 0, 0,0],
    [6*x2, 2, 0,0,0,0, 2*y2, 0, 0,0],
    [6*x3, 2, 0,0,0,0, 2*y3, 0, 0,0],
    ])

matrixC = np.array([z[x_idx4, y_idx4], partial_derivatives1[x_idx1, y_idx1,0], partial_derivatives1[x_idx2, y_idx2,0], partial_derivatives1[x_idx3, y_idx3,0], partial_derivatives1[x_idx1, y_idx1, 1], partial_derivatives1[x_idx2, y_idx2, 1], partial_derivatives1[x_idx3, y_idx3, 1], partial_derivatives2[x_idx1, y_idx1,0], partial_derivatives2[x_idx2, y_idx2,0], partial_derivatives2[x_idx3, y_idx3,0]])

In [282]:
len(matrixC)

10

In [283]:
# numpy determinant
np.linalg.det(matrixA) 

-8.932613372897297e-21

In [284]:
coeff = np.dot(np.linalg.inv(matrixA), matrixC)

In [285]:
x_new = 4
y_new = 4
matrixB = np.array([x_new**3, x_new**2, x_new, y_new**3, y_new**2, y_new, x_new**2*y_new, x_new*y_new**2, x_new*y_new, 1])
np.dot(matrixB, coeff)

18.195744021169048

In [286]:
fn_(x_new,y_new)

0.5514266812416906

In [287]:
x_, y_ = np.meshgrid(x, y)
ztest = fn_(x_, y_)
spline = RectBivariateSpline(x_[0, :], y_[:,0], ztest)
spline(x_new,y_new)

array([[0.55138675]])

array([[-1.20046257]])

In [255]:
@njit
def cubic_spline_interpolator(xnew, coefficients, x):
    # Handling extrapolation
    i = np.searchsorted(x, xnew) - 1
    idx1 = xnew <= x[0]
    idx2 = xnew > x[-1]
    i[idx1] = 0
    i[idx2] = len(x) - 2

    # Calculate the relative position within the interval
    dx = xnew - x[i]

    # Calculate the interpolated value
    # Cubic polynomial: a + b*dx + c*dx^2 + d*dx^3
    a, b, c, d = coefficients[:, i]
    #result = a + b*dx + c*dx**2 + d*dx**3
    result = d + c*dx + b*dx**2 + a*dx**3
    return result